**Install pycaret into environment**

In [ ]:
!pip install uv
!uv pip install pycaret[full] --system

Audited 1 package in 122ms


**Mount google drive and copy kaggle.json to machine**

In [ ]:
from google.colab import drive
drive.mount('mount')

Drive already mounted at mount; to attempt to forcibly remount, call drive.mount("mount", force_remount=True).
Drive already mounted at mount; to attempt to forcibly remount, call drive.mount("mount", force_remount=True).


In [ ]:
!cp mount/MyDrive/kaggle.json .
!ls

customer-personality-analysis.zip  logs.log		   mount
kaggle.json			   marketing_campaign.csv  sample_data
customer-personality-analysis.zip  kmeans_model.pkl  marketing_campaign.csv  sample_data
kaggle.json			   logs.log	     mount


**Acquire the customer personality dataset from kaggle and extract zip**


I am using this dataset uploaded 3 years ago on kaggle:

https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis

In [ ]:
!KAGGLE_CONFIG_DIR=$(pwd) kaggle datasets download imakash3011/customer-personality-analysis
!unzip -o customer-personality-analysis.zip

Dataset URL: https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis
License(s): CC0-1.0
customer-personality-analysis.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  customer-personality-analysis.zip
  inflating: marketing_campaign.csv  
Dataset URL: https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis
License(s): CC0-1.0
customer-personality-analysis.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  customer-personality-analysis.zip
  inflating: marketing_campaign.csv  


**Load the csv into a pandas dataframe**

In [ ]:
import pandas as pd
data = pd.read_csv('marketing_campaign.csv', sep='\t')
data.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


**Import pycaret and setup a clustering experiment**

In [ ]:
from pycaret.clustering import ClusteringExperiment
exp = ClusteringExperiment()

exp.setup(data, session_id = 1337, use_gpu=True)

,Description,Value
0,Session id,1337
1,Original data shape,"(2240, 29)"
2,Transformed data shape,"(2240, 702)"
3,Numeric features,26
4,Categorical features,3
5,Rows with missing values,1.1%
6,Preprocess,True
7,Imputation type,simple
8,Numeric imputation,mean
9,Categorical imputation,mode


**Train models**

In [ ]:
kmeans = exp.create_model('kmeans')

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.4934,4401.1951,0.4728,0,0,0


Processing:   0%|          | 0/3 [00:00<?, ?it/s]

**Evaluate Model**

In [ ]:
exp.evaluate_model(kmeans)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [ ]:
exp.plot_model(kmeans, plot = 'cluster')

**Assign model to label data**

In [ ]:
result = exp.assign_model(kmeans)
result.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Cluster
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,0,0,0,0,0,0,3,11,1,Cluster 1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,0,0,0,0,0,0,3,11,0,Cluster 1
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,0,0,0,0,0,0,3,11,0,Cluster 0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,0,0,0,0,0,0,3,11,0,Cluster 2
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,0,0,0,0,0,0,3,11,0,Cluster 1


**Predict on test data using Model**

In [ ]:
predictions = exp.predict_model(kmeans, data=data)
predictions.head()

,ID,Year_Birth,Education_Graduation,Education_PhD,Education_Master,Education_Basic,Education_2n Cycle,Marital_Status_Single,Marital_Status_Together,Marital_Status_Married,...,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Cluster
0,5524.0,1957.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,11.0,1.0,Cluster 1
1,2174.0,1954.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,11.0,0.0,Cluster 1
2,4141.0,1965.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,11.0,0.0,Cluster 0
3,6182.0,1984.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,11.0,0.0,Cluster 2
4,5324.0,1981.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,11.0,0.0,Cluster 1


**Save the model to disk**

In [ ]:
# save pipeline
exp.save_model(kmeans, 'kmeans_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['ID', 'Year_Birth', 'Income',
                                              'Kidhome', 'Teenhome', 'Recency',
                                              'MntWines', 'MntFruits',
                                              'MntMeatProducts',
                                              'MntFishProducts',
                                              'MntSweetProducts', 'MntGoldProds',
                                              'NumDealsPurchases',
                                              'NumWebPurchases',
                                              'NumCatalogPurchases',
                                              'NumStorePurchases',
                                              'NumWebVisitsMonth',
                                              'Acce...
                  TransformerWrapper(include=['Education', 'Marital_Status',
               